In [1]:
import glob, os, sys, re, random
from collections import defaultdict, Counter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ete3
from ete3 import PhyloTree
from Bio import SeqIO
import tqdm

PATH_TO_OUTPUT = '/home/gabs/Documents/lab/TermitesAndCockroaches/MutSpec-Redone/interim/ForDolphin/TermAndCock/midori_seqs_and_outgrp'

In [2]:
def collect_seqs(pattern: str, label: str, genes: list, sp=False):
    directory = "../../../MIDORI/"
    file = "MIDORI2_{}_NUC_SP_GB253_{}_BLAST.fasta" if sp else "MIDORI2_{}_NUC_GB253_{}_BLAST.fasta"
    path_to_fasta = directory + file
    seqs = defaultdict(list)
    for gene in tqdm.tqdm(genes):
        inp = path_to_fasta.format(label, gene)
        for rec in SeqIO.parse(inp, "fasta"):
            header = rec.description
            
            if pattern not in header:
                continue

            raw_acc, taxa = header.split("###")
            acc, place = re.match("(\w+\.\d)\.(.+)", raw_acc).groups()
            taxa = taxa.removeprefix("root_1;Eukaryota_2759;")
            species = " ".join(taxa.split(";")[-1].split("_")[:-1])
            rec.id = acc
            rec.description = f"{species} {place} ###{taxa}"
            seqs[gene + "__" + species.replace(" ", "_")].append(rec)
    return seqs


def filter_seqs(recs: list, var_gene_len=0.2, Nshare=0.2):
    assert isinstance(recs, list)
    seq_len_mean = np.median([len(x) for x in recs])
    min_seq_len = seq_len_mean * (1 - var_gene_len)
    max_seq_len = seq_len_mean * (1 + var_gene_len)

    recs_filtered = []
    for rec in recs:
        acgt_share = sum(rec.seq.count(x) for x in "ACGT") / len(rec)
        if min_seq_len < len(rec) < max_seq_len and 1 - acgt_share < Nshare:
            recs_filtered.append(rec)

    return recs_filtered


def filter_seqs_dict(seqs, nseqs_min=10, var_gene_len=0.2, Nshare=0.2):
    assert isinstance(seqs, dict)
    seqs_filtered = dict()
    for gene_species, recs in seqs.items():
        if len(recs) < nseqs_min:
            continue

        recs_filtered = filter_seqs(recs, var_gene_len, Nshare)
        if len(recs_filtered) >= nseqs_min:
            seqs_filtered[gene_species] = recs_filtered
    
    return seqs_filtered

## *Arthropoda*

In [3]:
genes = ['CO1']
seqs_arth = collect_seqs('Insecta', "UNIQ", genes)
len(seqs_arth)

100%|██████████| 1/1 [00:10<00:00, 10.37s/it]


120733

In [4]:
seqs_arth_filtered = filter_seqs_dict(seqs_arth)


In [39]:
def extract_sp_data(db_records):
    data = {} # all seqs and ids, family and genus for single species
    for sp in db_records:
        fams = []
        genuses = []
        seqs = []
        sps = []
        ids = []
        taxonomy = db_records[sp][0].description.split(';')
        fams.append(taxonomy[3])
        genuses.append(taxonomy[4])
        sps.append(taxonomy[5])
        for rec in range(len(db_records[sp])):
            id = db_records[sp][rec].id
            seq = db_records[sp][rec].seq
            seqs.append(seq)
            ids.append(id)
        data[sp] = [ids, sps, fams, genuses, seqs, db_records[sp][0].description]
    return(data)

def get_sp_and_outgrp(sp_data, db_records, path):
    #fams = []
    genuses = []
    ids = []

    for sp, stats in sp_data.items():
        #fams.append("".join(stats[2]))
        genuses.append("".join(stats[3]))
        ids.append(sp)

        with open(f'{path}/{sp}.fa', '+a') as fout:
            for i in range(len(stats[0])):
                fout.write(f'>{stats[0][i]} {stats[5]}\n{stats[4][i]}\n')


    genuses_stats = Counter(genuses)
    #fams_stats = Counter(fams)
    
    gene = []
    sp_ids = []
    outgrp_ids = []
    nseqs = []

    for i in range(len(ids)):
        #if genuses_stats[genuses[i]] == 1 and fams_stats[fams[i]] == 1:
        #    os.remove(f'{path}/{ids[i]}.fa')
        #elif genuses_stats[genuses[i]] == 1:
        #    for id, record in db_records.items():
        #        if fams[i] in record[0].name and id != ids[i]:
        #            with open(f'{path}/{ids[i]}.fa', 'a') as fout:
        #                fout.write(f'>OUTGRP\n{sp_data[id][4][0]} \n')
        #            break
        if genuses_stats[genuses[i]] == 1:
            os.remove(f'{path}/{ids[i]}.fa')
        elif genuses_stats[genuses[i]] > 1:
            for id, record in db_records.items():
                if genuses[i] in record[0].name and id != ids[i]:
                    sp_ids.append(ids[i].split("__")[1])
                    outgrp_ids.append(id.split("__")[1])
                    gene.append(id.split("__")[0])
                    nseqs.append(len(sp_data[ids[i]][4]))
                    with open(f'{path}/{ids[i]}.fa', '+a') as fout:
                        fout.write(f'>OUTGRP\n{sp_data[id][4][0]} \n')
                    break
    df = pd.DataFrame(zip(gene, sp_ids, nseqs, outgrp_ids), columns=['gene', 'species', 'nseqs', 'outgroup'])
    df.to_csv(f'{path}/1METADATA.csv')

In [40]:
data = extract_sp_data(seqs_arth_filtered)
get_sp_and_outgrp(data, seqs_arth_filtered, PATH_TO_OUTPUT)

## TESTING GROUNDS

In [35]:
test = {}
test['CO1__Heterolepisma_sclerophylla'] = seqs_arth_filtered['CO1__Heterolepisma_sclerophylla']
test['CO1__Lepisma_saccharina'] = seqs_arth_filtered['CO1__Lepisma_saccharina']
test['CO1__Zygaena_ephialtes'] = seqs_arth_filtered['CO1__Zygaena_ephialtes']
test['CO1__Zygaena_carniolica'] = seqs_arth_filtered['CO1__Zygaena_carniolica']
test['CO1__Zygaena_fausta'] = seqs_arth_filtered['CO1__Zygaena_fausta']
fams = []
genuses = []
ids = []

gene = []
sp_ids = []
outgrp_ids = []
nseqs = []

sp_data = extract_sp_data(test)
for sp, stats in sp_data.items():
    fams.append("".join(stats[2]))
    genuses.append("".join(stats[3]))
    ids.append(sp)
genuses_stats = Counter(genuses)
fams_stats = Counter(fams)
for i in range(len(ids)):
    if genuses_stats[genuses[i]] == 1:
        continue
    elif genuses_stats[genuses[i]] > 1:
        for id, record in test.items():
            if genuses[i] in record[0].name and id != ids[i]:
                sp_ids.append(ids[i].split("__")[1])
                outgrp_ids.append(id.split("__")[1])
                gene.append(id.split("__")[0])
                nseqs.append(len(sp_data[ids[i]][4]))
                break
df = pd.DataFrame(zip(gene, sp_ids, nseqs, outgrp_ids), columns=['gene', 'species', 'nseqs', 'outgroup'])
df.to_csv(f'{path}/1METADATA.csv')

16
